In [1]:
# infer GPU in use
!nvidia-smi

Mon Aug 19 13:28:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   21C    P0              50W / 400W |      0MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import torch
import numpy as np
import os, sys
import matplotlib.pyplot as plt

computer = 'argonne' # 'ucsc' or 'argonne'

# configure filepaths according to which computer we are on

if computer == 'ucsc':
    data_file = '/home/exx/shourya/comstock_data/G4601010_data.npz'
    weather_file = '/home/exx/shourya/comstock_data/G4601010_weather.npz'
    module_path = '/home/exx/shourya/time-series-forecasting-federation'
else:
    data_file = '/lcrc/project/NEXTGENOPT/NREL_COMSTOCK_DATA/grouped/G4601010_data.npz'
    weather_file = '/lcrc/project/NEXTGENOPT/NREL_COMSTOCK_DATA/grouped/G4601010_weather.npz'
    module_path = '/home/sbose/time-series-forecasting-federation'

# in this notebook, we will analyse the data files and try to see how we can make a custom data loader for the same.

data_y_s = np.load(data_file)
data_x_u = np.load(weather_file)

# function to calculate model sizes

def model_size_in_mb(model):
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())
    size_in_mb = param_size / (1024 ** 2)
    return f'{size_in_mb:.6f} MB'

# function to save model to disk and calculate its size

def save_and_measure_model(model):
    # Save state_dict
    torch.save(model.state_dict(), 'model_state.pth')

    # Measure file size
    file_size = os.path.getsize('model_state.pth') / (1024 * 1024)

    # Delete the file
    os.remove('model_state.pth')

    return f'{file_size:.6f} MB'
    
# base data type
base_type = torch.float32

In [3]:
# We now test out our dataset with california data

sys.path.insert(0,module_path)
from models.LFDataset import LFDataset

# create dataset
CA_dset = LFDataset(
    data_y_s = data_y_s,
    data_x_u = data_x_u,
    lookback = 100,
    lookahead = 4,
    client_idx = 0,
    idx_x = [0,1,2,3,4,5],
    idx_u = [6,7],
    dtype = base_type,
    normalize=True,
    normalize_type='z'
)

# load into dataloader
from torch.utils.data import DataLoader
CA_dataloader = DataLoader(CA_dset, batch_size = 32, shuffle = True)

<class 'numpy.ndarray'>
with_mean: True
with_std: True
copy: True
n_features_in_: 1
n_samples_seen_: 7
mean_: [11273.82960991]
var_: [6.77648757e+08]
scale_: [26031.68755852]



with_mean: True
with_std: True
copy: True
n_features_in_: 6
n_samples_seen_: 28032
mean_: [  8.62998778   4.53433665 182.47452911  67.67972317 202.4050371
 181.32121793]
var_: [1.90457134e+02 7.10934475e+00 6.77019366e+04 8.74746816e+03
 9.36539076e+04 1.00097295e+04]
scale_: [ 13.80062079   2.66633545 260.19595808  93.52790045 306.02925941
 100.04863583]



with_mean: True
with_std: True
copy: True
n_features_in_: 2
n_samples_seen_: 28032
mean_: [47.5         2.98305508]
var_: [767.91666667   3.98241121]
scale_: [27.71130936  1.99559796]



with_mean: True
with_std: True
copy: True
n_features_in_: 1
n_samples_seen_: 7
mean_: [11273.82960991]
var_: [6.77648757e+08]
scale_: [26031.68755852]


In [4]:
# Test out the shape of the dataloader outputs

for cidx, (a,b) in enumerate(CA_dataloader):
    print(f"Shape of {cidx+1}th item in a is {a.shape}.")
    print(f"Shape of {cidx+1}th item in b is {b.shape}.")
    break

l = a[0]
print(l.shape)
def save_array_to_csv(array, column_titles, filename):
    assert array.shape[1] == len(column_titles), "Column titles length must match array's second dimension"
    header = ','.join(column_titles)
    np.savetxt(filename, array, delimiter=',', header=header, comments='')
names = [
    'load',
    'dry bulb temp',
    'wind speed',
    'global horizontal radiation',
    'diffuse horizontal radiation',
    'direct normal radiation',
    'wind direction',
    'time index',
    'weekday index',
    'floor area',
    'wall area',
    'window area',
    'number of spaces',
    'number of zones',
    'number of surfaces',
    'cooling equipment capacity'
]
save_array_to_csv(l[:,:16],names,'client0_100rows.csv')

TypeError: LFDataset._transform() missing 1 required positional argument: 'y_tar'

In [ ]:
# Ensure that relative imports from the git repository can always be found

import sys
sys.path.insert(0,module_path)

# kwargs for all models
model_kwargs = {
    'x_size': 6,
    'y_size': 1,
    'u_size': 2,
    's_size': 7,
    'lookback': 12,
    'lookahead': 4
}

In [ ]:
# test out LSTM autoregressive version

import torch
import torch.nn as nn
from models.LSTM.LSTMAR import LSTMAR

model = LSTMAR(
    **model_kwargs
)
model_name = 'LSTM AR'

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of {model_name} output is {tuple(w.shape)}\n")
    break

# print model size
print(f"{model_name}, {base_type}, theoretical: {model_size_in_mb(model)}")
print(f"{model_name}, {base_type}, state_dict on disk: {model_size_in_mb(model)}")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = LSTMAR(
    **model_kwargs,
    dtype=dtype
).to(device)

print(f"{model_name}, {dtype}, theoretical: {model_size_in_mb(model2)}")
print(f"{model_name}, {dtype}, state_dict on disk: {model_size_in_mb(model2)}")

# evaluate the model
print("\n----\nTESTING IO DATA TYPES FOR torch.float{16|64}.\n----\n")
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of LSTM AR output is (32, 4, 1)

LSTM AR, torch.float32, theoretical: 0.167164 MB
LSTM AR, torch.float32, state_dict on disk: 0.167164 MB
LSTM AR, torch.float16, theoretical: 0.083582 MB
LSTM AR, torch.float16, state_dict on disk: 0.083582 MB

----
TESTING IO DATA TYPES FOR torch.float{16|64}.
----

Input dtype is torch.float16.
Output dtype is device is torch.float16.


In [ ]:
# test out DARNN

import torch
import torch.nn as nn
from models.DARNN.DARNN import DARNN

model = DARNN(
    **model_kwargs
)
model_name = 'DARNN'

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of {model_name} output is {tuple(w.shape)}\n")
    break

# print model size
print(f"{model_name}, {base_type}, theoretical: {model_size_in_mb(model)}")
print(f"{model_name}, {base_type}, state_dict on disk: {model_size_in_mb(model)}")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = DARNN(
    **model_kwargs,
    dtype=dtype
).to(device)

print(f"{model_name}, {dtype}, theoretical: {model_size_in_mb(model2)}")
print(f"{model_name}, {dtype}, state_dict on disk: {model_size_in_mb(model2)}")

# evaluate the model
print("\n----\nTESTING IO DATA TYPES FOR torch.float{16|64}.\n----\n")
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of DARNN output is (32, 4, 1)

DARNN, torch.float32, theoretical: 0.057205 MB
DARNN, torch.float32, state_dict on disk: 0.057205 MB
DARNN, torch.float16, theoretical: 0.028603 MB
DARNN, torch.float16, state_dict on disk: 0.028603 MB

----
TESTING IO DATA TYPES FOR torch.float{16|64}.
----

Input dtype is torch.float16.
Output dtype is device is torch.float16.


In [ ]:
# test out Transformer

import torch
import torch.nn as nn
from models.TRANSFORMER.Transformer import Transformer

model = Transformer(
    **model_kwargs
)
model_name = 'Transformer'

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of {model_name} output is {tuple(w.shape)}\n")
    break

# print model size
print(f"{model_name}, {base_type}, theoretical: {model_size_in_mb(model)}")
print(f"{model_name}, {base_type}, state_dict on disk: {model_size_in_mb(model)}")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = Transformer(
    **model_kwargs,
    dtype=dtype
).to(device)

print(f"{model_name}, {dtype}, theoretical: {model_size_in_mb(model2)}")
print(f"{model_name}, {dtype}, state_dict on disk: {model_size_in_mb(model2)}")

# evaluate the model
print("\n----\nTESTING IO DATA TYPES FOR torch.float{16|64}.\n----\n")
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of Transformer output is (32, 4, 1)

Transformer, torch.float32, theoretical: 2.116947 MB
Transformer, torch.float32, state_dict on disk: 2.116947 MB
Transformer, torch.float16, theoretical: 1.058474 MB
Transformer, torch.float16, state_dict on disk: 1.058474 MB

----
TESTING IO DATA TYPES FOR torch.float{16|64}.
----

Input dtype is torch.float16.
Output dtype is device is torch.float16.


In [ ]:
# test out Logtrans

import torch
import torch.nn as nn
from models.LOGTRANS.LogTrans import LogTrans
model = LogTrans(
    **model_kwargs
)
model_name = 'LogTrans'

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of {model_name} output is {tuple(w.shape)}\n")
    break

# print model size
print(f"{model_name}, {base_type}, theoretical: {model_size_in_mb(model)}")
print(f"{model_name}, {base_type}, state_dict on disk: {model_size_in_mb(model)}")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = LogTrans(
    **model_kwargs,
    dtype=dtype
).to(device)

print(f"{model_name}, {dtype}, theoretical: {model_size_in_mb(model2)}")
print(f"{model_name}, {dtype}, state_dict on disk: {model_size_in_mb(model2)}")

# evaluate the model
print("\n----\nTESTING IO DATA TYPES FOR torch.float{16|64}.\n----\n")
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of LogTrans output is (32, 4, 1)

LogTrans, torch.float32, theoretical: 1.419682 MB
LogTrans, torch.float32, state_dict on disk: 1.419682 MB
LogTrans, torch.float16, theoretical: 0.709841 MB
LogTrans, torch.float16, state_dict on disk: 0.709841 MB

----
TESTING IO DATA TYPES FOR torch.float{16|64}.
----

Input dtype is torch.float16.
Output dtype is device is torch.float16.


In [ ]:
# test out Informer

import torch
import torch.nn as nn
from models.INFORMER.Informer import Informer
model = Informer(
    **model_kwargs
)
model_name = 'Informer'

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of {model_name} output is {tuple(w.shape)}\n")
    break

# print model size
print(f"{model_name}, {base_type}, theoretical: {model_size_in_mb(model)}")
print(f"{model_name}, {base_type}, state_dict on disk: {model_size_in_mb(model)}")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = Informer(
    **model_kwargs,
    dtype=dtype
).to(device)

print(f"{model_name}, {dtype}, theoretical: {model_size_in_mb(model2)}")
print(f"{model_name}, {dtype}, state_dict on disk: {model_size_in_mb(model2)}")

# evaluate the model
print("\n----\nTESTING IO DATA TYPES FOR torch.float{16|64}.\n----\n")
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of Informer output is (32, 4, 1)

Informer, torch.float32, theoretical: 1.467289 MB
Informer, torch.float32, state_dict on disk: 1.467289 MB
Informer, torch.float16, theoretical: 0.733644 MB
Informer, torch.float16, state_dict on disk: 0.733644 MB

----
TESTING IO DATA TYPES FOR torch.float{16|64}.
----

Input dtype is torch.float16.
Output dtype is device is torch.float16.


In [ ]:
# test out Autoformer

import torch
import torch.nn as nn
from models.AUTOFORMER.Autoformer import Autoformer
model = Autoformer(
    **model_kwargs
)
model_name = 'Autoformer'

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of {model_name} output is {tuple(w.shape)}\n")
    break

# print model size
print(f"{model_name}, {base_type}, theoretical: {model_size_in_mb(model)}")
print(f"{model_name}, {base_type}, state_dict on disk: {model_size_in_mb(model)}")

# create model of a different dtype
dtype, device = torch.float64, 'cuda'
print(f"\nNOTICE: {model_name} does not support torch.float16 due to torch.fft.rfft not working with Half for all input tensor shapes.\n")
model2 = Autoformer(
    **model_kwargs,
    dtype=dtype
).to(device)

print(f"{model_name}, {dtype}, theoretical: {model_size_in_mb(model2)}")
print(f"{model_name}, {dtype}, state_dict on disk: {model_size_in_mb(model2)}")

# evaluate the model
print("\n----\nTESTING IO DATA TYPES FOR torch.float{16|64}.\n----\n")
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of Autoformer output is (32, 4, 1)

Autoformer, torch.float32, theoretical: 1.407627 MB
Autoformer, torch.float32, state_dict on disk: 1.407627 MB

NOTICE: Autoformer does not support torch.float16 due to torch.fft.rfft not working with Half for all input tensor shapes.

Autoformer, torch.float64, theoretical: 2.815254 MB
Autoformer, torch.float64, state_dict on disk: 2.815254 MB

----
TESTING IO DATA TYPES FOR torch.float{16|64}.
----

Input dtype is torch.float64.
Output dtype is device is torch.float64.


In [ ]:
# test out Fedformer Wavelet

import torch
import torch.nn as nn
from models.FEDFORMER.FedformerWavelet import FedformerWavelet
model = FedformerWavelet(
    **model_kwargs
)
model_name = 'Fedformer Wavelet'

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of {model_name} output is {tuple(w.shape)}\n")
    break

# print model size
print(f"{model_name}, {base_type}, theoretical: {model_size_in_mb(model)}")
print(f"{model_name}, {base_type}, state_dict on disk: {model_size_in_mb(model)}")

# create model of a different dtype
dtype, device = torch.float64, 'cuda'
print(f"\nNOTICE: {model_name} does not support torch.float16 due to torch.fft.rfft not working with Half for all input tensor shapes.\n")
model2 = FedformerWavelet(
    **model_kwargs,
    dtype=dtype
).to(device)

print(f"{model_name}, {dtype}, theoretical: {model_size_in_mb(model2)}")
print(f"{model_name}, {dtype}, state_dict on disk: {model_size_in_mb(model2)}")

# evaluate the model
print("\n----\nTESTING IO DATA TYPES FOR torch.float{16|64}.\n----\n")
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of Fedformer Wavelet output is (32, 4, 1)

Fedformer Wavelet, torch.float32, theoretical: 770.922871 MB
Fedformer Wavelet, torch.float32, state_dict on disk: 770.922871 MB

NOTICE: Fedformer Wavelet does not support torch.float16 due to torch.fft.rfft not working with Half for all input tensor shapes.

Fedformer Wavelet, torch.float64, theoretical: 1541.845467 MB
Fedformer Wavelet, torch.float64, state_dict on disk: 1541.845467 MB

----
TESTING IO DATA TYPES FOR torch.float{16|64}.
----

Input dtype is torch.float64.
Output dtype is device is torch.float64.


In [ ]:
# test out Fedformer Fourier

import torch
import torch.nn as nn
from models.FEDFORMER.FedformerFourier import FedformerFourier
model = FedformerFourier(
    **model_kwargs
)
model_name = 'Fedformer Fourier'

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of {model_name} output is {tuple(w.shape)}\n")
    break

# print model size
print(f"{model_name}, {base_type}, theoretical: {model_size_in_mb(model)}")
print(f"{model_name}, {base_type}, state_dict on disk: {model_size_in_mb(model)}")

# create model of a different dtype
dtype, device = torch.float64, 'cuda'
print(f"\nNOTICE: {model_name} does not support torch.float16 due to torch.fft.rfft not working with Half for all input tensor shapes.\n")
model2 = FedformerFourier(
    **model_kwargs,
    dtype=dtype
).to(device)

print(f"{model_name}, {dtype}, theoretical: {model_size_in_mb(model2)}")
print(f"{model_name}, {dtype}, state_dict on disk: {model_size_in_mb(model2)}")

# evaluate the model
print("\n----\nTESTING IO DATA TYPES FOR torch.float{16|64}.\n----\n")
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of Fedformer Fourier output is (32, 4, 1)

Fedformer Fourier, torch.float32, theoretical: 1.468189 MB
Fedformer Fourier, torch.float32, state_dict on disk: 1.468189 MB

NOTICE: Fedformer Fourier does not support torch.float16 due to torch.fft.rfft not working with Half for all input tensor shapes.

Fedformer Fourier, torch.float64, theoretical: 2.936378 MB
Fedformer Fourier, torch.float64, state_dict on disk: 2.936378 MB

----
TESTING IO DATA TYPES FOR torch.float{16|64}.
----

Input dtype is torch.float64.
Output dtype is device is torch.float64.


In [ ]:
# test out Crossformer

import torch
import torch.nn as nn
from models.CROSSFORMER.Crossformer import Crossformer
model = Crossformer(
    **model_kwargs
)
model_name = 'Crossformer'

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of {model_name} output is {tuple(w.shape)}\n")
    break

# print model size
print(f"{model_name}, {base_type}, theoretical: {model_size_in_mb(model)}")
print(f"{model_name}, {base_type}, state_dict on disk: {model_size_in_mb(model)}")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = Crossformer(
    **model_kwargs,
    dtype=dtype
).to(device)

print(f"{model_name}, {dtype}, theoretical: {model_size_in_mb(model2)}")
print(f"{model_name}, {dtype}, state_dict on disk: {model_size_in_mb(model2)}")

# evaluate the model
print("\n----\nTESTING IO DATA TYPES FOR torch.float{16|64}.\n----\n")
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of Crossformer output is (32, 4, 1)

Crossformer, torch.float32, theoretical: 3.905296 MB
Crossformer, torch.float32, state_dict on disk: 3.905296 MB
Crossformer, torch.float16, theoretical: 1.952648 MB
Crossformer, torch.float16, state_dict on disk: 1.952648 MB

----
TESTING IO DATA TYPES FOR torch.float{16|64}.
----

Input dtype is torch.float16.
Output dtype is device is torch.float16.


In [ ]:
# test out Crossformer

import torch
import torch.nn as nn
from models.XLSTM.mLSTM import mLSTM
model = mLSTM(
    **model_kwargs
)
model_name = 'mLSTM'

# evaluate the model
for a,b in CA_dataloader:
    w = model(a)
    print(f"Shape of {model_name} output is {tuple(w.shape)}\n")
    break

# print model size
print(f"{model_name}, {base_type}, theoretical: {model_size_in_mb(model)}")
print(f"{model_name}, {base_type}, state_dict on disk: {model_size_in_mb(model)}")

# create model of a different dtype
dtype, device = torch.float16, 'cuda'
model2 = mLSTM(
    **model_kwargs,
    dtype=dtype
).to(device)

print(f"{model_name}, {dtype}, theoretical: {model_size_in_mb(model2)}")
print(f"{model_name}, {dtype}, state_dict on disk: {model_size_in_mb(model2)}")

# evaluate the model
print("\n----\nTESTING IO DATA TYPES FOR torch.float{16|64}.\n----\n")
for a,b in CA_dataloader:
    a,b = a.to(dtype).to(device), b.to(dtype).to(device)
    w = model2(a)
    print(f"Input dtype is {dtype}.")
    print(f"Output dtype is device is {w.dtype}.")
    break

Shape of mLSTM output is (32, 4, 1)

mLSTM, torch.float32, theoretical: 1.063274 MB
mLSTM, torch.float32, state_dict on disk: 1.063274 MB
mLSTM, torch.float16, theoretical: 0.531637 MB
mLSTM, torch.float16, state_dict on disk: 0.531637 MB

----
TESTING IO DATA TYPES FOR torch.float{16|64}.
----

Input dtype is torch.float16.
Output dtype is device is torch.float16.
